# Utah State income tax

PolicyEngine models Utah's State income tax, including the following components:
* The main rate applied to taxable income (currently 4.85%)
* The (nonrefundable) taxpayer credit
* The income tax exemption

In [10]:
from policyengine_us import Simulation
import plotly.express as px
import pandas as pd


def create_situation(num_dependents, has_spouse):
    situation = {
        "people": {
            "person": {
                "age": 30,
            },
        },
        "tax_units": {
            "tax_unit": {
                "members": ["person"],
            },
        },
        "households": {
            "household": {
                "state_code": "UT",
                "members": ["person"],
            },
        },
        "axes": [
            [
                {
                    "name": "employment_income",
                    "min": 0,
                    "max": 200_000,
                    "count": 100,
                }
            ]
        ],
    }
    for i in range(num_dependents):
        situation["people"][f"person{i}"] = {"age": 10}
        situation["tax_units"]["tax_unit"]["members"].append(f"person{i}")
        situation["households"]["household"]["members"].append(f"person{i}")
    if has_spouse:
        situation["people"]["spouse"] = {"age": 30}
        situation["tax_units"]["tax_unit"]["members"].append("spouse")
        situation["households"]["household"]["members"].append("spouse")
    return situation


def create_simulation(situation):
    return Simulation(
        situation=situation,
    )


single_no_dependents = create_simulation(create_situation(0, False))
single_with_dependents = create_simulation(create_situation(1, False))
married_no_dependents = create_simulation(create_situation(0, True))
married_with_dependents = create_simulation(create_situation(1, True))

employment_income = single_no_dependents.calculate("employment_income")

# Create a dataframe with

df = pd.concat(
    [
        pd.DataFrame(
            {
                "Employment income": employment_income,
                "Utah income tax": simulation.calculate("ut_income_tax"),
                "Household type": household_type,
            }
        )
        for simulation, household_type in [
            (single_no_dependents, "Single, no dependents"),
            (single_with_dependents, "Single, with 1 dependent"),
            (married_no_dependents, "Married, no dependents"),
            (married_with_dependents, "Married, with 1 dependent"),
        ]
    ]
)
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

px.line(
    df,
    x="Employment income",
    y="Utah income tax",
    color="Household type",
    color_discrete_map={
        "Single, no dependents": GRAY,
        "Single, with 1 dependent": BLUE,
        "Married, no dependents": LIGHT_BLUE,
        "Married, with 1 dependent": DARK_BLUE,
    },
    template="plotly_white",
).update_layout(
    title="Utah income tax by household type",
    xaxis_title="Employment income",
    yaxis_title="Utah income tax",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
)